In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [4]:
# read file
news = pd.read_csv("https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv", index_col = 0)
news.head(3)

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary
0,141,awm.com,unreliable,http://awm.com/church-congregation-brings-gift...,Sometimes the power of Christmas will make you...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Church Congregation Brings Gift to Waitresses ...,Ruth Harris,NaN,[''],NaN,NaN,NaN
1,256,beforeitsnews.com,fake,http://beforeitsnews.com/awakening-start-here/...,AWAKENING OF 12 STRANDS of DNA – “Reconnecting...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,AWAKENING OF 12 STRANDS of DNA – “Reconnecting...,Zurich Times,NaN,[''],NaN,NaN,NaN
2,700,cnnnext.com,unreliable,http://www.cnnnext.com/video/18526/never-hike-...,Never Hike Alone: A Friday the 13th Fan Film U...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Never Hike Alone - A Friday the 13th Fan Film ...,NaN,NaN,[''],Never Hike Alone: A Friday the 13th Fan Film ...,NaN,NaN


In [11]:
# preprocess
news2 = news.drop_duplicates('content', 'first')
df = pd.DataFrame(news2, columns = ['id', 'domain', 'type', 'url', 'content', 'scraped_at', 'title', 'authors', 'meta_keywords', 'tags'])
df.isnull().sum()

id                 0
domain             0
type              12
url                0
content            0
scraped_at         0
title              0
authors           75
meta_keywords      0
tags             212
dtype: int64

In [14]:
df = df.fillna({'type': 'uncategorized','authors': 'Anonymous', 'tags': 'unknown'})
df.isnull().sum()

id               0
domain           0
type             0
url              0
content          0
scraped_at       0
title            0
authors          0
meta_keywords    0
tags             0
dtype: int64

In [9]:
import re
import nltk

In [19]:
'''
# Regexp for date
monthsShort = "Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec"
monthsLong = "January|February|March|April|May|June|July|August|September|October|November|December"
months = "(" + monthsShort + "|" + monthsLong + ")"
separators = "[-/\s,.]"
days = "\d{2}"
years = "\d{4}"
regex1 = "(" + months + separators + days + "|" + years + ")"
regex2 = "(" + days + separators + months + "|" + years + ")"
regex3 = '^([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])(\.|-|\/)([1-9]|0[1-9]|1[0-2])(\.|-|\/)([0-9][0-9]|19[0-9][0-9]|20[0-9][0-9])$|^([0-9][0-9]|19[0-9][0-9]|20[0-9][0-9])(\.|-|\/)([1-9]|0[1-9]|1[0-2])(\.|-|\/)([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])$'
'''

def my_clean_text(text):
    #replace multiple \s to single space and lower the text
    text = re.sub(r'(\\n)+|(\\t)+|\s{2,}', ' ', text).lower()
    #Replace email to <EMAIL>
    #text = re.sub(r'^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}$', ' <EMAIL> ', text)
    #Replace URLs to <URL>
    #text = re.sub(r'((http|ftp|https):\/\/)?[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?', ' <URL> ', text)
    #Replace date to <DATE>
    #text = re.sub(regex1, ' <DATE> ', text); text = re.sub(regex2, ' <DATE> ', text); text = re.sub(regex3, ' <DATE> ', text)
    #remove sign at start or end of the words
    text = re.sub(r'[^\w\s>]+\s|\s+[^\w\s<]', ' ', text)
    #remove in-words sign expect -,<,> and '
    text = re.sub(r'[^\w\s\-<>\']', '', text)
    #replace any numbers to <NUM>
    #text = re.sub(r'[\d+,?]\.?\d*', ' <NUM> ', text)
    #replace multiple \s to single space again
    text = re.sub(r'\s{2,}', ' ', text)
    #remove spical char
    #text = re.sub(r'["”“@()*|\'#!≥+.,$€%&"]', ' ', text)
    return text

# clean
df2 = df.copy(deep=True)
df2['content'] = df2.apply(lambda x: my_clean_text(x['content']), axis=1)
df2['title'] = df2.apply(lambda x: my_clean_text(x['title']), axis=1)

In [20]:
df2 = df2.rename(columns={'id':'art_id'})

In [30]:
df2.head()

,art_id,domain,type,url,content,scraped_at,title,authors,meta_keywords,tags,aut_id,source_id
0,141,awm.com,unreliable,http://awm.com/church-congregation-brings-gift...,sometimes the power of christmas will make you...,2018-01-25 16:17:44.789555,church congregation brings gift to waitresses ...,Ruth Harris,[''],unknown,-459550128026416448,-8705394394141738845
1,256,beforeitsnews.com,fake,http://beforeitsnews.com/awakening-start-here/...,awakening of 12 strands of dna reconnecting wi...,2018-01-25 16:17:44.789555,awakening of 12 strands of dna reconnecting wi...,Zurich Times,[''],unknown,-4164582980439435305,-1224964325638509372
2,700,cnnnext.com,unreliable,http://www.cnnnext.com/video/18526/never-hike-...,never hike alone a friday the 13th fan film us...,2018-01-25 16:17:44.789555,never hike alone a friday the 13th fan film fu...,Anonymous,[''],unknown,2581101060892613912,7449766137962270453
3,768,awm.com,unreliable,http://awm.com/elusive-alien-of-the-sea-caught...,when a rare shark was caught scientists were l...,2018-01-25 16:17:44.789555,elusive alien of the sea caught by scientist f...,Alexander Smith,[''],unknown,-6193758453579578776,-8705394394141738845
4,791,bipartisanreport.com,clickbait,http://bipartisanreport.com/2018/01/21/trumps-...,donald trump has the unnerving ability to abil...,2018-01-25 16:17:44.789555,trumps genius poll is complete the results hav...,Gloria Christie,[''],unknown,-1456891970558632779,-3391239009179761037


In [28]:
# add ID for table
df2['aut_id'] = df2.apply(lambda x: hash(x['authors']), axis=1)
df2['source_id'] = df2.apply(lambda x: hash(x['domain']), axis=1)

In [46]:
# write CSV
author = pd.DataFrame(df2, columns=['aut_id','authors'])
author = author.drop_duplicates('authors', 'first')
author.to_csv('/content/drive/MyDrive/Colab Notebooks/Project exercise/author.csv',header=True,index=False)

article = pd.DataFrame(df2, columns=['art_id', 'title', 'type', 'content', 'scraped_at','meta_keywords','tags'])
article = article.drop_duplicates('content', 'first')
article.to_csv('/content/drive/MyDrive/Colab Notebooks/Project exercise/article.csv',header=True,index=False)

source = pd.DataFrame(df2, columns=['source_id','url', 'domain','art_id'])
source = source.drop_duplicates('art_id', 'first')
source.to_csv('/content/drive/MyDrive/Colab Notebooks/Project exercise/source.csv',header=True,index=False)

#writes_for = pd.DataFrame(df2, columns=['aut_id', 'source_id'])
#writes_for = author.drop_duplicates('domain', 'first')
#writes_for.to_csv('/content/drive/MyDrive/Colab Notebooks/Project exercise/writes_for.csv',header=True,index=False)

#wrote = pd.DataFrame(df2, columns=['aut_id', 'art_id'])
#wrote = author.drop_duplicates('art_id', 'first')
#wrote.to_csv('/content/drive/MyDrive/Colab Notebooks/Project exercise/wrote.csv',header=True,index=False)

In [47]:
article.groupby('type').size().sort_values(ascending = False)

type
fake             146
conspiracy        31
political         22
uncategorized     12
unreliable         6
unknown            6
bias               6
junksci            5
reliable           3
hate               1
clickbait          1
dtype: int64